# American mass shootings Map visualizer

## Objectives

*   Showcase of year by year analysis of mass shootings in United States of America
*   Detailed visualization on location of the events, victims and the approximate timeline
*   Primary purpose is to engage user with fluid visuals to comprehand severity of these fatal events and spread awareness about gun violence
*   Data source : [Mother Jones and the Foundation for National Progress](https://docs.google.com/spreadsheets/d/e/2PACX-1vQBEbQoWMn_P81DuwmlQC0_jr2sJDzkkC0mvF6WLcM53ZYXi8RMfUlunvP1B5W0jRrJvH-wc-WGjDB1/pub?gid=0&single=true&output=csv)


In [1]:
import pandas as pd
import numpy as np
import geopy as gp
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline

In [51]:
# Reading the database
BaseData = pd.read_csv("Data/MotherJonesDataBase.csv")
BaseData['date'] = pd.to_datetime(BaseData['date'])
BaseData = BaseData.sort_values(by="date")
print(BaseData)

                                      case                location       date  \
129                  Welding shop shooting          Miami, Florida 1982-08-20   
128              Dallas nightclub shooting           Dallas, Texas 1984-06-29   
127         San Ysidro McDonald's massacre  San Ysidro, California 1984-07-18   
126  United States Postal Service shooting        Edmond, Oklahoma 1986-08-20   
125        Shopping centers spree killings       Palm Bay, Florida 1987-04-23   
..                                     ...                     ...        ...   
4        Sacramento County church shooting  Sacramento, California 2022-02-28   
3             Buffalo supermarket massacre       Buffalo, New York 2022-05-14   
2          Robb Elementary School massacre           Uvalde, Texas 2022-05-24   
1            Tulsa medical center shooting         Tulsa, Oklahoma 2022-06-01   
0                Concrete company shooting    Smithsburg, Maryland 2022-06-09   

                           

In [52]:
# Only fetching the values that is useful to us
BaseData = BaseData[['location','date','total_victims']]
print(BaseData)
BaseData.dtypes

                   location       date total_victims
129          Miami, Florida 1982-08-20            11
128           Dallas, Texas 1984-06-29             7
127  San Ysidro, California 1984-07-18            41
126        Edmond, Oklahoma 1986-08-20            21
125       Palm Bay, Florida 1987-04-23            20
..                      ...        ...           ...
4    Sacramento, California 2022-02-28             4
3         Buffalo, New York 2022-05-14            13
2             Uvalde, Texas 2022-05-24            38
1           Tulsa, Oklahoma 2022-06-01            TK
0      Smithsburg, Maryland 2022-06-09             4

[130 rows x 3 columns]


location                 object
date             datetime64[ns]
total_victims            object
dtype: object

In [53]:
# Making sure that everything is in correct format
BaseData['year'] = pd.DatetimeIndex(BaseData['date']).year
BaseData['month'] = pd.DatetimeIndex(BaseData['date']).month
# Dropping rows that has any missing values
BaseData = BaseData.dropna()
# Total_victim has to have all the numeric values, if not remove that
BaseData['total_victims'] = pd.to_numeric(BaseData['total_victims'], errors = 'coerce')
BaseData = BaseData.dropna(subset=['total_victims'])

print(BaseData)

                   location  date  total_victims
129          Miami, Florida  1982           11.0
128           Dallas, Texas  1984            7.0
127  San Ysidro, California  1984           41.0
126        Edmond, Oklahoma  1986           21.0
125       Palm Bay, Florida  1987           20.0
..                      ...   ...            ...
5          Oxford, Michigan  2021           11.0
4    Sacramento, California  2022            4.0
3         Buffalo, New York  2022           13.0
2             Uvalde, Texas  2022           38.0
0      Smithsburg, Maryland  2022            4.0

[129 rows x 3 columns]


In [55]:
# split the city and the state
BaseData[['city','state']] = BaseData['location'].str.split(',',expand=True)
# removing any extra spaces around the name of the city and state
BaseData['city'] = BaseData['city'].str.strip()
BaseData['state'] = BaseData['state'].str.strip()
BaseData.head()

,location,date,total_victims,city,state
129,"Miami, Florida",1982,11.0,Miami,Florida
128,"Dallas, Texas",1984,7.0,Dallas,Texas
127,"San Ysidro, California",1984,41.0,San Ysidro,California
126,"Edmond, Oklahoma",1986,21.0,Edmond,Oklahoma
125,"Palm Bay, Florida",1987,20.0,Palm Bay,Florida
